In [1]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### Static Variables

In [2]:
line = "18"

### Extracting Data

In [3]:
df = pd.read_csv("bus_data/line_data/" + line + ".csv", low_memory=False, header=None)
df.columns = ["Timestamp", "LineID", "JourneyPatternID", "TimeFrame", 
              "VehicleJourneyID", "Lon", "Lat", "VehicleID", "StopID", 
              "AtStop", "HumanTime", "Day", "Hour", "Runtime"]

#convert StopID to string

df['StopID'] = df['StopID'].astype('str')


df.shape

(81350, 14)

In [4]:
schedule = pd.read_csv("bus_data/dublinbus_scheduledData2013csv.csv", low_memory=False, encoding="ISO-8859-1")


### Exploring schedule

In [5]:
schedule.head()

,trip_id,arrival_time,departure_time,stop_id,Name without locality,Name,Lat,Long,stop_sequence,stop_headsign,shape_dist_traveled
0,1.1598.0-33A-b12-1.451.I,16:15:00,16:15:00,8240DB003813,Footbridge,Skerries Road,53.591377,-6.156176,1,Swords,0.000000
1,1.1598.0-33A-b12-1.451.I,16:15:39,16:15:39,8240DB003814,Barnageeragh Road,Balbriggan Rd,53.585544,-6.148605,2,Swords,827.070328
2,1.1598.0-33A-b12-1.451.I,16:17:11,16:17:11,8240DB003815,Barnageeragh Road,North Cliff Heights,53.581935,-6.125247,3,Swords,2746.649492
3,1.1598.0-33A-b12-1.451.I,16:17:22,16:17:22,8240DB003816,Mourne Drive,North Cliff Heights,53.580471,-6.127622,4,Swords,2973.900755
4,1.1598.0-33A-b12-1.451.I,16:17:44,16:17:44,8240DB003566,Selskar Court,Town Parks Rd,53.578067,-6.125055,5,Swords,3447.155571


In [6]:
# Extracting StopID from stop_id

def get_stop(string):
    var = string[-4:]
    return var

schedule["StopID"] = schedule['stop_id'].apply(lambda x: get_stop(x))

In [7]:
# Extracting LineID from trip_id

schedule["LineID"] = schedule["trip_id"].str.extract('\-(.*?)\-')

/Users/danieljordan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


In [8]:
# Number of unique LineIDs 

lines = schedule["LineID"].unique()

print(lines)

['33A' '41C' '114' '39' '25A' '25X' '39A' '75' '41' '13' '747' '9' '76'
 '53' '151' '104' '27' '63' '44' '123' '11' '4' '16' '16C' '1' '61' '15'
 '236' '84' '185' '38' '70' '14' '38A' '14C' '40' '40B' '83' '38B' '83A'
 '18' '45A' '29A' '31' '31A' '32' '7' '7D' '150' '145' '184' '46A' '47'
 '140' '41A' '43' '42' '65B' '65' '41B' '33' '27B' '102' '33B' '17A' '25B'
 '122' '25' '17' '59' '67' '120' '238' '270' '27A' '56A' '77A' '37' '26'
 '66' '66A' '220' '130' '40D' '15B' '15A' '54A' '49' '79A' '79' '68' '69'
 '116' '161' '41X' '239' '68A' '69X' '66X' '51D' '8' '7B' '33X' '66B' '67X'
 '76A' '27X' '44B' '86' '332' '84A' '51X' '46E' '31B' '111' '32X' '142'
 '84X' '118']


In [9]:
# Isolating line
# Examining how many stops are on each variation
# Saving this info in stops

schedule_15 = schedule[schedule.LineID == line]

headsigns = schedule_15.stop_headsign.unique()

stops = {}

for sign in headsigns:
    stop_count = schedule_15[schedule_15.stop_headsign == sign].StopID.nunique()
    stop_list = schedule_15[schedule_15.stop_headsign == sign].StopID.unique()
    stops[sign] = [stop_count, stop_list]

### Exploring line 15

In [10]:
# # Isolating variations - getting variation with most stops

# df_0001 = df[df.JourneyPatternID == 1]
# df_1001 = df[df.JourneyPatternID == 1001]
# df_0002 = df[df.JourneyPatternID == 2]
# df_1002 = df[df.JourneyPatternID == 1002]

df_var = df.groupby.JourneyPatternID


In [11]:
# Naming variations

df_0001.name = "df_0001"
df_1001.name = "df_1001"
df_0002.name = "df_0002"
df_1002.name = "df_1002"



In [12]:
runtimes = df_0001.groupby("StopID").Runtime.mean()

In [13]:
# print(runtimes.sort_values())

In [15]:
# print("1218" in stops["Ballycullen Road"][1])

In [16]:
# stoplist = [x for x in df_0002.StopID.unique() if x not in stops["Ballycullen Road"][1]]

In [17]:
# print(stoplist)

### Comparing Schedule to our data


In [18]:
# Number of unique stops on each variation in our data

print(df_0001.StopID.nunique())
print(df_1001.StopID.nunique())
print(df_0002.StopID.nunique())
print(df_1002.StopID.nunique())

71
72
0
50


In [19]:
# Number of stops on each variation in schedule - doesn't line up exactly 

for key, value in stops.items():
    print(key, ":", value[0])

Palmerstown : 62
Sandymount : 64


In [ ]:
# # Testing how many stops in our data are missing from schedule

# journey_patterns = [df_0001, df_1001, df_1002, df_0002]

# for pattern in journey_patterns:
#     set1 = set(pattern.StopID.unique())
#     print(len(set1.difference(set(schedule_stops))))


In [ ]:
# Finding stops that are on schedule variation but not our data

set1 = set(df_0001.StopID.unique())
set2 = set(stops["Ballycullen Road"][1])

misc_stops = set2.difference(set1)

print(list(misc_stops))

In [ ]:
# Looking at number of shared stops in our data & schedule

for pattern in journey_patterns:
    set1 = set(pattern.StopID.unique())
    print("Number of stops in", pattern.name, "=", len(set1))
    for key, value in stops.items():
        set2 = set(value[1])
#         print(pattern.name, "&", key, ":", set2.issubset(set1))
        print(pattern.name, "&", key, ":", len(set2.intersection(set1)))
    print()

# print(set1.isdisjoint(set2))
# print(set1.difference(set2))

### Plotting Stops

In [ ]:
# Plotting stops in our data

locations = df_0001[["Lon", "Lat"]]
locations.plot.scatter(x="Lon", y="Lat")

In [ ]:
# Plotting stops in schedule
variation_stops = stops["Ballycullen Road"][1]

schedule_locations = schedule_15[schedule_15.StopID.isin(variation_stops)]
schedule_locations = schedule_15[["Lat", "Long"]]
schedule_locations.plot.scatter(x="Long", y="Lat")

In [ ]:
# Plotting stops in schedule but not data (misc_stops)

misc_df = schedule_15[schedule_15.StopID.isin(misc_stops)]

misc_df.head()

locations = misc_df[["Long", "Lat"]]
locations.plot.scatter(x="Long", y="Lat")

### Removing irrelevant stops in our data

In [ ]:
# df_0001["StopID"].value_counts()

In [ ]:
# Removing stops that don't occur at least 100 times in our data

groups = df_0001.groupby('StopID')
clean_0001 = groups.filter(lambda x: len(x) > 100)

In [ ]:
clean_0001["StopID"].value_counts()

In [ ]:
# Plotting stops in our data

locations = clean_0001[["Lon", "Lat"]]
locations.plot.scatter(x="Lon", y="Lat")

In [ ]:
df_0001["StopID"].value_counts().plot()

### Plotting common routes between data and schedule

In [ ]:
set1 = set(df_0001.StopID.unique())
set2 = set(stops["Ballycullen Road"][1])

common_stops = set2.intersection(set1)

In [ ]:
# Plotting stops in schedule and data (common_stops)

common_df = schedule_15[schedule_15.StopID.isin(common_stops)]

common_df.head()

locations = common_df[["Long", "Lat"]]
locations.plot.scatter(x="Long", y="Lat")